In [ ]:
# ==========================================================
# RoBERTa / XLM-RoBERTa para FRAUDE (Colab compatible)
# ✔ Sin evaluation_strategy (API antigua)
# ✔ Entrena, evalúa, ajusta umbral y prueba mensajes
# ==========================================================

!pip -q install transformers datasets sentencepiece scikit-learn

import numpy as np
import pandas as pd
import torch

from google.colab import files
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    set_seed
)

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_recall_fscore_support,
    confusion_matrix,
    roc_auc_score,
    average_precision_score,
    classification_report
)

import matplotlib.pyplot as plt

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# -----------------------------
# 1) Subir dataset
# -----------------------------
print("Sube el dataset etiquetado (dataset_etiquetado_v2.csv)")
uploaded = files.upload()
data_path = list(uploaded.keys())[0]

df = pd.read_csv(data_path)

TEXT_COL = "Mensaje"
LABEL_COL = "fraud_label"

df[TEXT_COL] = df[TEXT_COL].fillna("").astype(str)
df[LABEL_COL] = df[LABEL_COL].astype(int)

print("\nDistribución etiquetas:")
print(df[LABEL_COL].value_counts())

# -----------------------------
# 2) Split estratificado
# -----------------------------
train_df, test_df = train_test_split(
    df[[TEXT_COL, LABEL_COL]],
    test_size=0.2,
    random_state=42,
    stratify=df[LABEL_COL]
)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
test_ds  = Dataset.from_pandas(test_df.reset_index(drop=True))

# -----------------------------
# 3) Modelo (usar XLM-RoBERTa: estable en Colab)
# -----------------------------
MODEL_NAME = "xlm-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
).to(device)

# -----------------------------
# 4) Tokenización
# -----------------------------
MAX_LEN = 256

def tokenize(batch):
    return tokenizer(batch[TEXT_COL], truncation=True, max_length=MAX_LEN)

train_tok = train_ds.map(tokenize, batched=True)
test_tok  = test_ds.map(tokenize, batched=True)

train_tok = train_tok.rename_column(LABEL_COL, "labels")
test_tok  = test_tok.rename_column(LABEL_COL, "labels")

train_tok = train_tok.remove_columns([TEXT_COL])
test_tok  = test_tok.remove_columns([TEXT_COL])

data_collator = DataCollatorWithPadding(tokenizer)

# -----------------------------
# 5) TrainingArguments (API antigua compatible)
# -----------------------------
args = TrainingArguments(
    output_dir="roberta_out",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=10_000,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# -----------------------------
# 6) Entrenar
# -----------------------------
trainer.train()

# -----------------------------
# 7) Evaluación manual (TEST)
# -----------------------------
pred = trainer.predict(test_tok)

logits = pred.predictions
labels = pred.label_ids

probs = torch.softmax(torch.tensor(logits), dim=1).numpy()
p1 = probs[:, 1]

# -----------------------------
# 8) Ajuste fino de umbral
# -----------------------------
def eval_thr(y, p, t):
    y_pred = (p >= t).astype(int)
    pr, rc, f1, _ = precision_recall_fscore_support(
        y, y_pred, average="binary", zero_division=0
    )
    return pr, rc, f1, y_pred

thresholds = np.arange(0.05, 0.96, 0.01)
rows = []
for t in thresholds:
    pr, rc, f1, _ = eval_thr(labels, p1, t)
    rows.append((t, pr, rc, f1))

thr_df = pd.DataFrame(rows, columns=["threshold","precision","recall","f1"])
best = thr_df.loc[thr_df["f1"].idxmax()]
best_thr = float(best["threshold"])

best_p, best_r, best_f1, y_pred = eval_thr(labels, p1, best_thr)

roc_auc = roc_auc_score(labels, p1)
pr_auc  = average_precision_score(labels, p1)
cm = confusion_matrix(labels, y_pred)

# -----------------------------
# 9) Resultados
# -----------------------------
print("\n=== Umbral óptimo ===")
print(best)

print("\n=== Métricas (TEST) ===")
print(f"Precision: {best_p:.4f}")
print(f"Recall   : {best_r:.4f}")
print(f"F1-score : {best_f1:.4f}")
print(f"ROC-AUC  : {roc_auc:.4f}")
print(f"PR-AUC   : {pr_auc:.4f}")

print("\n=== Classification report ===")
print(classification_report(labels, y_pred, digits=4))

print("=== Matriz de confusión ===")
print(cm)

# -----------------------------
# 10) Modo prueba interactivo
# -----------------------------
def predict_message(msg, threshold=best_thr):
    enc = tokenizer(msg, return_tensors="pt", truncation=True, max_length=MAX_LEN).to(device)
    with torch.no_grad():
        out = model(**enc).logits
        prob = torch.softmax(out, dim=1)[0,1].item()
    return prob, int(prob >= threshold)

print("\n=== MODO PRUEBA ===")
print("Escribe un mensaje | 'salir' para terminar\n")

while True:
    msg = input("Mensaje: ")
    if msg.lower() in ["salir", "exit", "quit"]:
        break
    prob, pred = predict_message(msg)
    print("Predicción:", "FRAUDE" if pred else "NO FRAUDE")
    print("Probabilidad:", round(prob,4))
    print("-"*50)


Device: cuda
Sube el dataset etiquetado (dataset_etiquetado_v2.csv)


Saving dataset_etiquetado_v2.csv to dataset_etiquetado_v2 (3).csv

Distribución etiquetas:
fraud_label
1    1795
0    1490
Name: count, dtype: int64


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2628 [00:00<?, ? examples/s]

Map:   0%|          | 0/657 [00:00<?, ? examples/s]

/tmp/ipython-input-3796202954.py:115: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,0.597500
200,0.406700
300,0.380100
400,0.324100
500,0.327400
600,0.337100
700,0.287800
800,0.224200
900,0.237900



=== Umbral óptimo ===
threshold    0.740000
precision    0.894595
recall       0.922006
f1           0.908093
Name: 69, dtype: float64

=== Métricas (TEST) ===
Precision: 0.8946
Recall   : 0.9220
F1-score : 0.9081
ROC-AUC  : 0.9637
PR-AUC   : 0.9751

=== Classification report ===
              precision    recall  f1-score   support

           0     0.9024    0.8691    0.8855       298
           1     0.8946    0.9220    0.9081       359

    accuracy                         0.8980       657
   macro avg     0.8985    0.8956    0.8968       657
weighted avg     0.8982    0.8980    0.8978       657

=== Matriz de confusión ===
[[259  39]
 [ 28 331]]

=== MODO PRUEBA ===
Escribe un mensaje | 'salir' para terminar

Predicción: FRAUDE
Probabilidad: 0.9997
--------------------------------------------------
